https://huggingface.co/docs/peft/quicktour

关闭多线程分词，避免多进程冲突警告。

Hugging Face 的 tokenizers 库在后台使用 Rust 多线程加速（并行分词）。
但是，当 Python 的 multiprocessing 在已经启用了并行的情况下调用 fork() 时，会触发这个警告，以避免潜在的死锁。

In [ ]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"  # 禁用分词器并行警告
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"  # 允许 MPS 不支持的算子自动 fallback 到 CPU

In [ ]:
import torch
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

In [ ]:
import numpy as np

## 一、使用 bigscience/mt0-large

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "bigscience/mt0-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
model = model.to(device)

inputs = tokenizer("Translate English to Chinese: Hello, how are you?", return_tensors="pt")
outputs = model.generate(**inputs)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


## 二、微调训练

每个 PEFT 方法都由一个 PeftConfig 类定义，该类存储构建 PeftModel 所需的所有重要参数。

例如，要使用 LoRa 进行训练，请加载并创建一个 LoraConfig 类。

- task_type ：要训练的任务（本例中为序列到序列语言建模）
- inference_mode ：是否使用该模型进行推理
- r ：低秩矩阵的维数
- lora_alpha ：低秩矩阵的缩放因子
- lora_dropout ：LoRa 层的 dropout 概率

In [ ]:
from peft import LoraConfig, TaskType

peft_config = LoraConfig(task_type=TaskType.SEQ_2_SEQ_LM, inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1)
#peft_config = LoraConfig(task_type="SEQ_2_SEQ_LM", r=4, lora_alpha=8, lora_dropout=0.05)

LoraConfig 设置完成后，使用 get_peft_model() 函数创建一个 PeftModel 。

该函数接受一个基础模型（可以从 Transformers 库加载）和一个 LoraConfig， 其中包含用于配置模型以使用 LoRA 进行训练的参数。

In [ ]:
from peft import get_peft_model

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

现在你可以使用 Transformers Trainer 、Accelerate 或任何自定义的 PyTorch 训练循环来训练模型了。

例如，要使用 Trainer 类进行训练，请设置一个 TrainingArguments 类，并设置一些训练超参数。

加载一个翻译数据集，英文到中文

In [ ]:
from datasets import load_dataset
dataset = load_dataset("opus100", "en-zh", split="train[:1%]").train_test_split(test_size=0.1)

In [ ]:
dataset

In [ ]:
dataset["train"][:3]

对数据进行预处理

In [ ]:
# 定义预处理函数：把原始文本构造成给模型的 prompt，并进行分词
def preprocess(examples):
    # 构造输入 prompt（mt0 是 instruction-style，因此加上任务说明）   
    inputs = [f"translate English to Chinese: {ex['en']}" for ex in examples["translation"]] # 把每个英文句子包装成翻译指令
    targets = [ex["zh"] for ex in examples["translation"]]

    # 使用 tokenizer 对 inputs 做编码（截断 + padding 到 max_length）
    model_inputs = tokenizer(inputs, truncation=True, padding="max_length", max_length=128)  # 编码输入，统一长度

    # 对目标文本进行编码，得到 labels
    labels = tokenizer(targets, truncation=True, padding="max_length", max_length=128)  # 编码目标文本

    # 将 labels 的 pad token 替换为 -100，这是 transformers 损失函数忽略的标记
    # 这样在计算交叉熵时，对 pad 部分不会贡献损失
    labels_input_ids = [
        [(token if token != tokenizer.pad_token_id else -100) for token in label] for label in labels["input_ids"]
    ]  # 将 labels 中的 pad id 转为 -100

    # 把处理好的 labels 放回 model_inputs 字典，Trainer 会读取 "labels" 字段用于计算 loss
    model_inputs["labels"] = labels_input_ids  # 为返回的数据添加 labels

    # 返回最终的编码后的输入字典（包含 input_ids, attention_mask, labels）
    return model_inputs  # 返回每个样本的字典

# 把预处理函数映射到整个数据集（batched=True 表示按批次处理以加速）
tokenized_datasets = dataset.map(preprocess, batched=True)  # 对 dataset 的每个 split 应用 preprocess 函数

In [ ]:
tokenized_datasets

In [ ]:
example = tokenized_datasets["train"][0]
print("Input:", tokenizer.decode(example["input_ids"], skip_special_tokens=True))
print("Label:", tokenizer.decode([id for id in example["labels"] if id != -100], skip_special_tokens=True))

加载评测指标

In [ ]:
import evaluate  # Hugging Face evaluate 库，用于加载评测指标
# 加载评测指标（示例使用 sacrebleu 用于机器翻译）
metric = evaluate.load("sacrebleu")  # 加载 sacrebleu 指标用于 BLEU 风格评测

In [ ]:
# 定义 compute_metrics，用于 Trainer 在 eval 时计算指标
def compute_metrics(eval_pred):
    # eval_pred 的典型形式是 (predictions, labels)；这里 predictions 是生成器输出的 token id 矩阵
    preds, labels = eval_pred  # 解包预测结果和标签

    # 如果 predictions 是 logits（非生成模式），则需要取 argmax；但 predict_with_generate=True 时 preds 已是 token id
    if isinstance(preds, tuple):  # 万一 preds 是 tuple（某些版本返回 logits 等），我们取第一个
        preds = preds[0]

    # 将预测中的 -100（如果有的话）替换为 tokenizer.pad_token_id，便于 decode
    # 但正常生成结果里不应该有 -100，这里是稳健处理
    preds = np.where(preds != -100, preds, tokenizer.pad_token_id)  # 将 -100 换为 pad id（以便 decode）

    # 把预测和真实标签解码成字符串（跳过 special tokens）
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)  # 解码预测文本
    # labels 里仍可能有 -100，需要把它们换成 pad_token_id 再解码
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)  # 将 labels 中的 -100 替回 pad id
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)  # 解码真实文本

    # sacrebleu 的 compute 需要 references 是 list of list 的形式：[[ref1], [ref2], ...]
    references = [[l] for l in decoded_labels]  # 为每个样本创建单一引用列表

    # 调用 sacrebleu 指标进行计算并返回结果字典（sacrebleu 返回 'score' 键表示 BLEU）
    result = metric.compute(predictions=decoded_preds, references=references)  # 计算 sacrebleu 分数
    # 将结果包装成 dict 返回，Trainer 会把它显示在 eval 输出里
    return {"bleu": result["score"]}  # 返回一个包含 BLEU 分数的字典

训练参数

In [ ]:
from transformers import Seq2SeqTrainingArguments
# ---- 训练参数 ----
training_args = Seq2SeqTrainingArguments(
    output_dir="./mt0-lora",                  # 模型输出和检查点保存目录
    per_device_train_batch_size=1,            # 每个 GPU/设备上的训练 batch size（根据显存调整）, # Mac上必须非常小，否则内存爆
    per_device_eval_batch_size=1,             # 验证时的 batch size
    gradient_accumulation_steps=8,            # # 模拟 batch size=8
    learning_rate=5e-5,                       # 学习率，默认5e-5，内存小调低学习率
    num_train_epochs=1,                       # 训练轮次，#先跑1个epoch测试稳定性
    eval_strategy="epoch",                    # 评估频率（这里设置为每个 epoch 评估一次）
    predict_with_generate=True,               # 在评估时使用 generate() 来产生文本（必需用于语言生成任务）
    save_strategy="epoch",                    # epoch 表示每个 epoch 保存一次checkpoint），# no 不保存中间检查点，节省内存
    logging_strategy="steps",                 # 日志策略（按 steps）
    logging_steps=100,                        # 每多少 step 打印一次日志
    fp16=False,                               # 如果支持则开启半精度训练以节省显存（需要 CUDA + 支持）#  MPS 不支持 fp16
    load_best_model_at_end=True,              # 在训练结束加载最佳验证结果对应的模型
    dataloader_pin_memory=False,              # 在 CUDA 上能让 DataLoader 更快地把数据拷贝到 GPU。MPS 不支持
    metric_for_best_model="bleu",             # 根据哪个指标判断最佳模型
)

创建训练器

In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model=model,                              # 注入（已包装 LoRA 的）模型
    args=training_args,                       # 训练参数
    train_dataset=tokenized_datasets["train"],# 训练集
    eval_dataset=tokenized_datasets["validation"] if "validation" in tokenized_datasets else tokenized_datasets["test"],  # 优先使用 validation split，否则用 test
    processing_class=tokenizer,     # tokenizer（用于 decode/generation 时）
    compute_metrics=compute_metrics,          # 评估时要计算的指标函数
)


In [ ]:
# ---- 启动训练 ----
trainer.train()  # 运行训练过程；训练期间会按 training_args 指定的策略保存 checkpoint 并在 eval 时计算指标

In [ ]:
# 9. 保存 LoRA 权重
trainer.save_model("./mt0-lora-mac-final")

# 测试生成的模型

In [ ]:
# 加载微调后模型
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from peft import PeftModel

model_name = "bigscience/mt0-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
base_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
model = PeftModel.from_pretrained(base_model, "./mt0-lora-mac-final")
model = model.to(device)

In [ ]:
# 测试生成
inputs = tokenizer("Translate English to Chinese: The difficult access to these older sites for displaced persons gives rise to concerns about the living conditions of the people who are residents there, in particular widows, the elderly and children, who often live in intolerable hardship.", return_tensors="pt")
outputs = model.generate(**inputs, max_new_tokens=1024)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
